<a href="https://colab.research.google.com/github/ImAliMoussa/Tensorflow-Speech-Recognition-Challenge/blob/main/TFSpeech_Recognition_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle

! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c tensorflow-speech-recognition-challenge

Saving kaggle.json to kaggle.json
 99% 1.03G/1.04G [00:10<00:00, 116MB/s] 
100% 1.04G/1.04G [00:11<00:00, 99.1MB/s]
  0% 0.00/501k [00:00<?, ?B/s]
100% 501k/501k [00:00<00:00, 161MB/s]
100% 2.45G/2.46G [00:51<00:00, 24.7MB/s]
100% 2.46G/2.46G [00:51<00:00, 51.2MB/s]
  0% 0.00/50.0 [00:00<?, ?B/s]
100% 50.0/50.0 [00:00<00:00, 43.9kB/s]


In [ ]:
! ls

kaggle.json		      sample_data	    test.7z
link_to_gcp_credits_form.txt  sample_submission.7z  train.7z


In [ ]:
! pip install -q tensorflow-io
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import librosa
import librosa.display

In [ ]:
! mkdir /kaggle
! mkdir /kaggle/working
%cd /content
! rm -rf /kaggle/working/train
! 7z x train.7z -o/kaggle/working/
! ls

/content

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1121103842 bytes (1070 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Blocks = 2

  0%      0% 39 - train/audio/_background_noise_/dude_miaowing.wav                                                            0% 41 - train/audio/_background_noise_/pink_noise.wav                                                         0% 42 - train/audio/_background_noise_/running_tap.wav

In [ ]:
%cd /kaggle/working/
! mv ./train/audio/_background_noise_/README.md .

/kaggle/working


In [ ]:
def load_test_file_set(test_file_path='./train/testing_list.txt'):
    #open the txt file containing the pathes of files that should be added to test dataset
    #convert the list to set datastructure
    test_files = open(test_file_path).read().splitlines()
    return set(test_files)

def load_validation_file_set(val_file_path='./train/validation_list.txt'):
    #open the txt file containing the pathes of files that should be added to validation dataset
    #convert the list to set datastructure
    validation_files = open(val_file_path).read().splitlines()
    return set(validation_files)

def is_test_file(test_set, file_path):
    return file_path in test_set

def is_validation_file(val_set, file_path):
    return file_path in val_set

In [ ]:
X_train = []
Y_train = []

X_val   = []
Y_val   = []
X_test  = []
Y_test  = []

training_files_pathes = []

Background_Noise = []
! rm -rf ./train/audio/silence
! mkdir ./train/audio/silence

def read_train_audio(folder_path='./train/audio', fraction_of_data=1):    
    global X_train, X_val, Y_train, Y_val, Background_Noise, training_files_pathes
    
    test_file_set = load_test_file_set()
    validation_file_set = load_validation_file_set()
    
    for label in os.listdir(folder_path): #loop on all folders in the audio folder
        dir_path = os.path.join(folder_path, label)
        if os.path.isfile(dir_path):#skip any files
            continue
        for filename in os.listdir(dir_path): #loop on all files in folder
            if label == '_background_noise_':
                full_file_path = os.path.join(dir_path, filename)
                audio = tfio.audio.AudioIOTensor(full_file_path).to_tensor().numpy()
                audio_len = audio.shape[0]
                to_remove = audio_len % 16000
                audio = audio[:-to_remove]
                Background_Noise.append(audio)
                
            else:
                full_file_path = os.path.join(dir_path, filename)
                if os.path.isdir(full_file_path):
                    continue
                file_rel_path = os.path.join(label, filename)
                if is_validation_file(validation_file_set, file_rel_path):
                    X_val.append(full_file_path)                    
                    Y_val.append(label)
                else:
                    X_train.append(full_file_path)                    
                    Y_train.append(label)

    Background_Noise = np.concatenate(Background_Noise, axis=0).reshape((-1, 16000, 1))
    from scipy.io.wavfile import write
    for i in range(len(Background_Noise)):
        samplerate = 16000
        fname = f'./train/audio/silence/silence{i}.wav'
        write(fname, samplerate, Background_Noise[i])
        if i % 3 == 0:
            for i in range(2):
                X_val.append(fname)
                Y_val.append('silence')
        else:
            # append several times to testing because homa 2olayelin
            for i in range(8):
                X_train.append(fname)
                Y_train.append('silence')


# to use all data read_train_audio(fraction_of_data=1)
read_train_audio()

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

def encode_strings():
    global label_encoder, Y_train, Y_val
    label_encoder.fit(Y_train)
    print('classes : ', list(label_encoder.classes_))
    Y_train = label_encoder.transform(Y_train)
    Y_val = label_encoder.transform(Y_val)
    
encode_strings()

classes :  ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [ ]:
print('X Train (words + background noise) Dimensions:', len(X_train))
print('Y Train' , Y_train.shape)

print('X Validation' , len(X_val))
print('Y Validation' , Y_val.shape)

print('Background noise' , Background_Noise.shape)

X Train (words + background noise) Dimensions: 59403
Y Train (59403,)
X Validation 6984
Y Validation (6984,)
Background noise (278, 16000, 1)


In [ ]:
mx = -1e9
mn = 1e9
for i in range(len(X_train)):
    if Y_train[i] != 'silence':
        # print(X_train[i])
        s = tfio.audio.AudioIOTensor(X_train[i]).to_tensor().numpy().reshape((-1, 1))
        curmax = np.max(s)
        curmin = np.min(s)
        mx = max(mx, curmax)
        mn = min(mn, curmin)
print(mn, mx)

-32768 32767


In [ ]:
X_train[-10:]

['./train/audio/silence/silence275.wav',
 './train/audio/silence/silence275.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav',
 './train/audio/silence/silence277.wav']

In [ ]:
def get_SR_Model2(num_classes: int):


  ################################################################
    #best hyper params:
        # conv_1  (128, 31, strides=8) --> Dropout = 0.2
        # conv_2  (256, 31, strides=8) --> Dropout = 0.2
        # lstm_   (512, return_sequences=False) --> Dropout = 0.2
        # dense_1 (512)  --> Dropout = 0.4
  ################################################################


    X_input = Input(shape=(16000, 1))
    
    X = Conv1D(128, 31, strides=7, name='conv_1')(X_input) 
    X = BatchNormalization()(X)
    X = Activation('relu', name='conv1_relu')(X)
    X = Dropout(0.3)(X)
    
    X = Conv1D(256, 31, strides=7, name='conv_2')(X)
    X = BatchNormalization()(X)
    X = Activation('relu', name='conv2_relu')(X)
    X = Dropout(0.3)(X)
    
    X = GRU(units=512, return_sequences=False, name='gru')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.3)(X)
    
    X = Dense(512, activation='relu', name='dense')(X)
    X = BatchNormalization()(X)
    X = Dropout(0.3)(X)
    
    X = Dense(num_classes, activation='softmax', name='output')(X)

    return Model(inputs=[X_input], outputs=[X])

In [ ]:
import math

class SpeechSequence(tf.keras.utils.Sequence):
    def __init__(self, data, labels, batch_size=64, shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        X = np.zeros(shape=(64, 16000, 1))
        for i in range(len(indexes)):
            audio = tfio.audio.AudioIOTensor(self.data[indexes[i]]).to_tensor().numpy().reshape((-1, 1))
            X[i, 0 : len(audio)] += audio
            # X[i] += Background_Noise[np.random.randint(len(Background_Noise))]
        y = self.labels[indexes]
        return X, y


In [ ]:
# train_seq = SpeechSequence(data = X_train, labels = Y_train)
# valid_seq = SpeechSequence(data = X_val, labels = Y_val)

model = get_SR_Model2(31)
# model.compile(loss=losses.sparse_categorical_crossentropy,
#               optimizer='adam',
#               metrics=['acc'])
# print(model.summary())

# checkpoint_filepath = '/tmp/checkpoint'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
#                                         save_weights_only=True,
#                                         monitor='val_acc',
#                                         mode='max',
#                                         save_best_only=True)

# history = model.fit(x=train_seq, validation_data=valid_seq, epochs=35, callbacks=[model_checkpoint_callback])

In [ ]:
classes = [0 for i in range(35)]
for i in Y_train:
    classes[i] += 1
for i in range(len(classes)):
    print(i, classes[i])

0 1516
1 1569
2 1565
3 1576
4 2095
5 2109
6 2115
7 2092
8 2112
9 1553
10 1577
11 2106
12 1586
13 2134
14 2105
15 2101
16 2110
17 2140
18 2111
19 2114
20 1558
21 1480
22 2107
23 2134
24 2108
25 1567
26 2137
27 2115
28 1579
29 2116
30 2116
31 0
32 0
33 0
34 0


In [ ]:
classes = [0 for i in range(35)]
for i in Y_val:
    classes[i] += 1
for i in range(len(classes)):
    print(i, classes[i])

0 197
1 162
2 168
3 170
4 264
5 243
6 242
7 280
8 260
9 189
10 173
11 247
12 160
13 230
14 270
15 256
16 257
17 230
18 256
19 263
20 176
21 186
22 262
23 246
24 248
25 166
26 236
27 260
28 166
29 261
30 260
31 0
32 0
33 0
34 0


In [ ]:
model.load_weights('/tmp/checkpoint')

In [ ]:
ls /tmp/checkpoint

/tmp/checkpoint


In [ ]:
! ls /content/drive/'My Drive'
! cp -r /tmp/checkpoint  /content/drive/'My Drive'

'Colab Notebooks'   test


In [ ]:
X_test = open("./testing_files.txt").read().splitlines()

In [ ]:
X_test[0:5]

['clip_000044442.wav',
 'clip_0000adecb.wav',
 'clip_0000d4322.wav',
 'clip_0000fb6fe.wav',
 'clip_0001d1559.wav']

In [ ]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O test

--2020-10-12 16:23:18--  https://docs.google.com/uc?export=download&id=1ZZQC05zomH3iEVgPSaITj8BDamusejM7
Resolving docs.google.com (docs.google.com)... 74.125.142.138, 74.125.142.139, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.138|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-10-12 16:23:18 ERROR 404: Not Found.



In [ ]:
Y_test = []
dirname = "/content/drive/My Drive/test/audio/"
counter = 0
print(len(X_test))
for filename in X_test:
    counter += 1
    x = np.zeros(shape=(1, 16000, 1))
    audio = tfio.audio.AudioIOTensor(dirname + filename).to_tensor().numpy().reshape((-1, 1))
    x[0, 0:len(audio)] = audio
    Y_test.append(np.argmax(model.predict(x)))
    if counter % 50 == 0:
        print(counter)


75451
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400


KeyboardInterrupt: ignored

In [ ]:
model.save("my_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets


In [ ]:
! ls /content/drive/'My Drive'
# ! ls my_model/
# ! zip -r my_model.zip my_model

 checkpoint  'Colab Notebooks'	 model_checkpoint   my_model   test
